# Natural Language Processing

En lineamientos generales este proyecto es sumamente similar al proyecto pasado, pero enfocado a trabajar con texto. Este mini proyecto estará dividido en las mismas 4 secciones que el proyecto anterior.

[Dataset Glue - MRPC](https://huggingface.co/datasets/nyu-mll/glue)

![Vista previa del dataset](https://github.com/ichcanziho/Deep_Learnining_Platzi/raw/master/4%20Curso%20de%20Transfer%20Learning%20con%20Hugging%20Face/imgs%2F4%20Natural%20Language%20Processing%20%2F1.png)

El tema de `MRPC` es dada dos oraciones `sentence1, sentence2` etiquetar si ambos textos son equivalentes o no lo son. Cómo `Feature_Extractor` y `Pretrained Model` vamos a usar [distilroberta-base](https://huggingface.co/distilbert/distilroberta-base) el cual es un modelo más pequeño y rápido que un modelo BERT convencional.


## Carga de datasets para NLP


In [2]:
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

In [3]:
ds = load_dataset("nyu-mll/glue", "mrpc")

Una vez descargado el dataset podemos observar que su estructura es la siguiente:


In [4]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


Nuestro dataset contiene **3668** ejemplos de entrenamiento, **408** ejemplos de validación y **1725** ejemplos de prueba. Cada uno de ellos contiene 2 oraciones una etiqueta y un índice.


Mostremos un ejemplo del subset de `train`


In [5]:
# mostremos un ejemplo:

ejemplo = ds["train"][400]
print(ejemplo)

{'sentence1': 'U.S. Agriculture Secretary Ann Veneman , who announced Tuesdays ban , also said Washington would send a technical team to Canada to help .', 'sentence2': "U.S. Agriculture Secretary Ann Veneman , who announced yesterday 's ban , also said Washington would send a technical team to Canada to assist in the Canadian situation .", 'label': 1, 'idx': 446}


Recordemos que a nosotros como humanos nos gusta leer etiquetas con un nombre y no con un numero, entonces conozcamos cuáles son los nombres de las `labels`:


In [6]:
labels = ds["train"].features["label"]
print(labels)

ClassLabel(names=['not_equivalent', 'equivalent'], id=None)


Excelente, ya sabemos que los datos pueden ser **not_equivalent** o **equivalent**, afortunadamente, el objeto labels cuenta con un simple método int2str que convierte un número a su representación de texto, entonces nuestro ejemplo que tiene un label 1 podemos acceder al nombre de la etiqueta de la siguiente manera:


In [7]:
# Nombre de la etiqueta

label_name = labels.int2str(ejemplo["label"])
print(label_name)

equivalent


Perfecto, hasta este momento ya estamos familiarizados con el dataset que estaremos utilizando y vemos como hasta este punto el proyecto es sumamente parecido al proyecto pasado, continuemos en la siguiente clase.


## Procesamiento de dataset para NLP

Al igual que en el ejemplo de `Computer Vision`, se debe hacer un preprocesamiento para trabajar con el texto de entrada de modo que tenga la forma correcta que está buscando nuestro modelo `pre-entrenado` específicamente en `NLP` El texto NO puede ser clasificado directamente, debe ser convertido en una representación numerica para poder trabajar con ellos. A este proceso de transformación se le conoce como: `Word Embeddings` Para conocer más acerca de este tema te recomiendo ver el siguiente video: Intro al NLP ¿Qué es un EMBEDDING?. Ya hemos trabajado con este concepto en otros cursos anteriores de esta ruta de Deep Learning si quieres leer un poco más al respecto te recomiendo ver: Variables Estadísticas


En general el objetivo de este paso es pre-procesar los datos de texto de nuestro dataset en una representación vectorial que sea compatible con el modelo que vas a usar. Para ello vamos a usar el propio `Tokenizer` del modelo que vamos a utilizar en este proyecto: Primero, tengamos en cuenta que vamos a usar el modelo **distilroberta-base**, vamos a utilizar `AutoTokenizer` para obtener el tokenizador correspondiente a nuestro modelo:


In [8]:
base_model_url = "distilbert/distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(base_model_url)

ejemplo = ds["train"][400]["sentence1"]
print(ejemplo)
print("\nTokenizado: ")
print(tokenizer(ds["train"][400]["sentence1"]))

U.S. Agriculture Secretary Ann Veneman , who announced Tuesdays ban , also said Washington would send a technical team to Canada to help .

Tokenizado: 
{'input_ids': [0, 791, 4, 104, 4, 8004, 1863, 3921, 10336, 5649, 2156, 54, 585, 20423, 7033, 2020, 2156, 67, 26, 663, 74, 2142, 10, 3165, 165, 7, 896, 7, 244, 479, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


### ¿Qué significa cada uno de los valores que nos retorna el tokenizador?

- `input_ids` es la traducción de palabras a números.

- `attention_mask` es un tensor con la misma forma que input_ids, pero lleno de 0 y 1: los 1 indican que se debe atender a los tokens correspondientes y los 0 indican que no se deben atender. Es decir, deben ser ignorados por el modelo.

- `token_type_ids` dice al modelo qué parte de la entrada es la primera oración y cuál es la segunda oración.

Existen otros modelos más complejos como: `"bert-base-uncased"` cuyo **tokenizador** contiene un método especializado en transformar de regreso los `inputs_ids` en el texto original: `tokenizer.convert_ids_to_tokens(inputs["input_ids"])` En este caso, nuestro modelo ligero NO contiene dicho método, pero es bueno tener en cuenta que otros modelos sí tienen esta capacidad.


**Creemos una función Tokenizadora:**


In [9]:
def tokenize_fn(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

Afortunadamente, nuestro objeto `tokenizer` permite recibir multiples `parámetros` separados por `,` esto me permite transformar nuestras 2 oraciones de ejemplo al mismo tiempo. El parámetro `truncatoion = True` significa que oraciones sumamente largas serán recortadas a unas más pequeñas de un tamaño predefinido por el modelo.

Finalmente, vamos a `limpiar` todo nuestro dataset, a diferencia del dataset de imágenes que ocupábamos el método `ds.with_transform(transform)` y utilizábamos una función `transform` para modificar el dataset, como específicamente podemos transformar nuestros datos de entrada a su versión tokenizada con `tokenize_fn` podemos usar el método map para mapear los parámetros de `sentence1` y `sentence2` de cada muestra del dataset


In [10]:
clean_ds = ds.map(tokenize_fn, batched=True)
print(clean_ds["train"][400])

{'sentence1': 'U.S. Agriculture Secretary Ann Veneman , who announced Tuesdays ban , also said Washington would send a technical team to Canada to help .', 'sentence2': "U.S. Agriculture Secretary Ann Veneman , who announced yesterday 's ban , also said Washington would send a technical team to Canada to assist in the Canadian situation .", 'label': 1, 'idx': 446, 'input_ids': [0, 791, 4, 104, 4, 8004, 1863, 3921, 10336, 5649, 2156, 54, 585, 20423, 7033, 2020, 2156, 67, 26, 663, 74, 2142, 10, 3165, 165, 7, 896, 7, 244, 479, 2, 2, 791, 4, 104, 4, 8004, 1863, 3921, 10336, 5649, 2156, 54, 585, 2350, 128, 29, 2020, 2156, 67, 26, 663, 74, 2142, 10, 3165, 165, 7, 896, 7, 3991, 11, 5, 1563, 1068, 479, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


```JSON
{
'sentence1': 'U.S. Agriculture Secretary Ann Veneman , who announced Tuesdays ban , also said Washington would send a technical team to Canada to help .',
'sentence2': "U.S. Agriculture Secretary Ann Veneman , who announced yesterday 's ban , also said Washington would send a technical team to Canada to assist in the Canadian situation .",
'label': 1,
'idx': 446,
'input_ids': [0, 791, 4, 104, 4, 8004, 1863, 3921, 10336, 5649, 2156, 54, 585, 20423, 7033, 2020, 2156, 67, 26, 663, 74, 2142, 10, 3165, 165, 7, 896, 7, 244, 479, 2, 2, 791, 4, 104, 4, 8004, 1863, 3921, 10336, 5649, 2156, 54, 585, 2350, 128, 29, 2020, 2156, 67, 26, 663, 74, 2142, 10, 3165, 165, 7, 896, 7, 3991, 11, 5, 1563, 1068, 479, 2],
'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
}
```


Excelente, ahora nuestro diccionario de una muestra de entrenamiento contiene `6 keys` las 4 originales y 2 adicionales, `input_ids` y `attention_mask`

Finalmente, a pesar de que él `dataset` ya está `limpio` al igual que en el ejemplo de `Computer Vision` necesitamos definir un Data Collactor


## Definiendo el data collator: Dynamic padding

Necesitamos que nuestros tensores tengan una forma rectangular. Es decir que tengan el mismo tamaño cada uno de los ejemplos. Sin embargo, los textos no necesariamente tienen el mismo tamaño.

Para ello usamos el relleno o padding. El padding se asegura de que todas nuestras oraciones tengan la misma longitud al agregar una palabra especial llamada padding token a las oraciones con menos valores. Por ejemplo, si tenemos 10 oraciones con 10 palabras y 1 oración con 20 palabras, el relleno garantizará que todas las oraciones tengan 20 palabras.

Dejamos el argumento de padding del tokenizer vacío en nuestra función de tokenización por ahora. Esto se debe a que rellenar (hacer padding) todas las muestras hasta la longitud máxima del dataset no es eficiente, es mejor rellenar las muestras cuando estamos construyendo un batch, ya que entonces solo necesitamos rellenar hasta la longitud máxima en ese batch, y no la longitud máxima en todo el dataset. ¡Esto puede ahorrar mucho tiempo y potencia de procesamiento cuando las entradas tienen longitudes muy variables!

Usaremos un DataCollator para esto. Rellenemos (hagamos padding) todos los ejemplos con la longitud del elemento más largo del batch. A esta técnica se le conoce como relleno dinámico o dynamic padding.


In [17]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
print(data_collator)

DataCollatorWithPadding(tokenizer=RobertaTokenizerFast(name_or_path='distilbert/distilroberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=N

Perfecto, ya hemos definido exitosamente a nuestro `DataCollator` esto nos permite definir a nuestro `Trainer` pero esto lo veremos en la siguiente clase.


## Configurando un Trainer para NLP

Esta biblioteca nos va a ayudar con el mismo objetivo que lo hizo `from datasets import load_metric` pero en este caso, la métrica que vamos a utilizar va a estar definida por el propio dataset, en el curso no queda muy claro porque usar está metodología en lugar de la habitual.

En general observamos que la estructura de la función `compute_metrics` es sumamente similar a la que ya conociamos, solo que en lugar de usar `load_metric` estamos usando `evaluate.load()`, en general me pareció más entendible la forma pasada.


In [18]:
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels_ = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels_)

A continuación vamos a crear a nuestro modelo desde un `pretrained` como el modelo es una clasificación de texto, lo debemos importar usando `AutoModelForSquencueClassification`, es importante conocer `transformers` para conocer todas las clases que podemos utilizar para cargar modelos pre-entrenados, pero en este curso ya vimos las 2 más populares.


Más adelante cuando hayamos terminado el entrenado del modelo, veremos cuáles son las métricas, y observaremos que el método de evaluate.load() cargo dos métricas ACC y F1 score, algo sumamente interesante y curioso que vale la pena poner como nota.


In [19]:
model = AutoModelForSequenceClassification.from_pretrained(
    base_model_url,
    num_labels=len(labels.names),
    id2label={str(i): c for i, c in enumerate(labels.names)},
    label2id={c: str(i) for i, c in enumerate(labels.names)},
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ahora podemos definir nuestros argumentos de entrenamiento:


In [20]:
training_args = TrainingArguments(
    output_dir="../M1K3DEV23-distilroberta-base-mrpc-glue",
    evaluation_strategy="steps",
    num_train_epochs=2,
    push_to_hub=False,
    load_best_model_at_end=True,
)

Esto fue sumamente parecido al problema de clasificación de imágenes, pero noto que aquí NO pusimos `learning_rate=2e-4, remove_unused_columns=False`, en la clase no se explica el porqué de este cambio.

Y para terminar con esta clase, vamos a definir a nuestro `trainer`:


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=clean_ds["train"],
    eval_dataset=clean_ds["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

c:\Users\mike\Documents\datos\transfer_learning_hf\venv\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Felicidades, ya tenemos TODO listo para empezar a entrenar a nuestro modelo, pero esto lo haremos en la siguiente clase.


### Entrenamiento y evaluación de modelo de NLP

Vamos a agregar las últimas líneas de código para entrenar al modelo y subirlo al HUB de Hugging Face y observar sus resultados de entrenamiento, evaluación y test.

Agregamos el código para entrenar el modelo:


In [22]:
# Entrenamos al modelo

train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)

  0%|          | 0/918 [00:00<?, ?it/s]

{'loss': 0.5411, 'grad_norm': 48.38636016845703, 'learning_rate': 2.2766884531590417e-05, 'epoch': 1.09}


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.6344608068466187, 'eval_accuracy': 0.803921568627451, 'eval_runtime': 15.8504, 'eval_samples_per_second': 25.741, 'eval_steps_per_second': 3.218, 'epoch': 1.09}
{'train_runtime': 1012.9986, 'train_samples_per_second': 7.242, 'train_steps_per_second': 0.906, 'train_loss': 0.4581408864272706, 'epoch': 2.0}
***** train metrics *****
  epoch                    =        2.0
  train_loss               =     0.4581
  train_runtime            = 0:16:52.99
  train_samples_per_second =      7.242
  train_steps_per_second   =      0.906


Finalmente evaluamos el modelo en el set de `test`:


In [23]:
# Evaluando en el conjunto de test

metrics = trainer.evaluate(clean_ds["test"])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

  0%|          | 0/216 [00:00<?, ?it/s]

***** eval metrics *****
  epoch                   =        2.0
  eval_accuracy           =     0.7884
  eval_loss               =     0.7235
  eval_runtime            = 0:01:03.39
  eval_samples_per_second =     27.212
  eval_steps_per_second   =      3.407


Felicidades, ya hemos terminado nuestro proyecto de NLP. Y hemos logrado obtener un accuracy de casi 80% en el set de pruebas
